In [11]:
!pip install -q openai langchain playwright beautifulsoup4 playwright install chroma chromadb tiktoken langchainhub

In [2]:
from getpass import getpass
import os
os.environ['OPENAI_API_KEY'] = getpass()

 ········


In [92]:
#Extract text from CDC website
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from bs4 import BeautifulSoup as Soup

import bs4
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
import time

#Send to GPT4 for cleanup
def clean_text_with_gpt4(text):
    """
    This function takes a string of text and uses GPT-4 to clean it up using the OpenAI ChatCompletion API.
    It handles large texts by breaking them into smaller chunks.
    :param text: String containing the text to be cleaned.
    :return: Cleaned text as a string.
    """
    cleaned_texts = []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
    chunks = text_splitter.split_documents(text)

    for chunk in chunks:
        try:
            response = client.chat.completions.create(
                model="gpt-4-0314",  # Assuming using the latest GPT-4 model
                messages=[{"role": "system", "content": "You are a helpful assistant."},
                          {"role": "user", "content": f"Please clean up the following text:\n\n{chunk}"}]
            )
            #print(response)
            cleaned_texts.append(response.choices[0].message.content.strip())
            #print(cleaned_texts)
            time.sleep(1)  # Delay to respect rate limits
        except Exception as e:
            print(f"An error occurred: {e}")
            return None
    return ' '.join(cleaned_texts)

file_name = "cleaned_texts.txt"
if os.path.exists(file_name):
    print(f"The file {file_name} already exists.")
    try:
        with open(file_name, 'r', encoding='utf-8') as file:
            content = file.read()
            #print("File content:\n")
            #print(content)
    except FileNotFoundError:
        print(f"The file {file_name} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    url = "https://www.cdc.gov"
    loader = RecursiveUrlLoader(
        url=url, max_depth=2, extractor=lambda x: Soup(x, "html.parser").text
    )
    docs = loader.load()
    
    client = OpenAI()
    cleaned_text = clean_text_with_gpt4(docs)
    with open(file_name, 'w', encoding='utf-8') as file:
        for text in cleaned_texts:
            file.write(text + "\n\n")  # Adding two newlines as a separator between texts
    print(f"Cleaned texts saved to {file_name}")

An error occurred: Error code: 404 - {'error': {'message': 'The model `gpt-4-0314` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}


NameError: name 'cleaned_texts' is not defined

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

text_splitter2 = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter2.split_documents(cleaned_text)

vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
CDC_retriever = vectorstore.as_retriever()

In [ ]:
import zipfile

file_name = "EpiHiper-Schema-master.zip"
extracted_folder = "extracted_files"

if os.path.exists(file_name):
    print(f"The file {file_name} already exists.")

    try:
        with zipfile.ZipFile(file_name, 'r') as zip_ref:
            zip_ref.extractall(extracted_folder)

        docs = []

        # Walk through the directory, including subdirectories
        for foldername, subfolders, filenames in os.walk(extracted_folder):
            for filename in filenames:
                file_path = os.path.join(foldername, filename)
                try:
                    with open(file_path, 'r', encoding='utf-8') as file:
                        docs.append(file.read())
                except Exception as e:
                    print(f"An error occurred while reading {filename}: {e}")

        # At this point, 'docs' contains the contents of all files from the zip
        # You can process 'docs' as needed

    except zipfile.BadZipFile:
        print("The file is not a zip file or it is corrupted.")
    except Exception as e:
        print(f"An error occurred: {e}")

code_splits = text_splitter2.split_documents(docs)
vectorstore = Chroma.from_documents(documents=code_splits, embedding=OpenAIEmbeddings())
ABM_retriever = vectorstore.as_retriever()

In [ ]:
from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=.2)
llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0.0) #"gpt-4-1106-preview", "gpt-4-0314"

query_planner = Agent(
    role="Simulation Planner",
    goal="Plan the steps needed to parameterize an agent-based simulation from existing knowledge",
    backstory=textwrap.dedent("""
        You are an expert at identifying modeling parameters from code base that implements 
        an agent-based model and listing the model choices, parameters, and json config files 
        whose values need to be determined. You will break down each model choice, parameter, 
        and json config file into sub-questions such that the answer to each sub-question will 
        inform the value to be used in the agent-based simulation.
        Accept the user-question and determine if it requires sub-questions to either the
        CDC website which provides an official source of recent infectious disease outbreaks
        or Wikipedia for information about a geographical location, country, infectious agent
        characteristics, transmission dynamics, infection states, or other epidemiological
        modeling efforts.
        Your final answer MUST be a description of sub-questions that explain the best model
        choices, model parameters, and config files for an agent-based modeling code base.
    """),
    verbose=True,
    allow_delegation=False,
    tools=[],  ###
    llm=llm,
)


In [ ]:
#from langchain import hub
#prompt = hub.pull("hwchase17/self-ask-with-search")
#print(f'{prompt.format(agent_scratchpad = "AGENTSCRATCHPAD", input = "INPUT")}')

from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import GoogleSearchAPIWrapper

CDC_retriever_tool = create_retriever_tool(
    CDC_retriever,
    "Intermediate Answer",
    """As an AI assistant you provide answers based on the given context, ensuring accuracy and briefness. 

        You always follow these guidelines:

        -If the answer isn't available within the context, state that fact
        -Otherwise, answer to your best capability, refering to source of documents provided
        -Only use examples if explicitly requested
        -Do not introduce examples outside of the context
        -Do not answer if context is absent
        -Limit responses to three or four sentences for clarity and conciseness
        
        Search for data related to outbreaks. For questions about outbreaks, use this tool to return 
        relevant data for answering questions about outbreaks""",
)

ABM_retriever_tool = create_retriever_tool(
    ABM_retriever,
    "Intermediate Answer",
    """As an AI assistant you provide answers based on the given context, ensuring accuracy and briefness. 

        You always follow these guidelines:

        -If the answer isn't available within the context, state that fact
        -Otherwise, answer to your best capability, refering to source of documents provided
        -Only use examples if explicitly requested
        -Do not introduce examples outside of the context
        -Do not answer if context is absent
        -Limit responses to three or four sentences for clarity and conciseness
        
        Search for data related to outbreaks. For questions about outbreaks, use this tool to return 
        relevant data for answering questions about outbreaks""",
)

search = GoogleSearchAPIWrapper()
Google_search_tool = Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or specialized information about a specific domain",
    )

query_executor = Agent(
    role="""
        Searcher of information from CDC for current outbreak information, from the code base
        for agent-based modeling simulation parameters and settings, or from Wikipedia for disease 
        specific information such a transmission, vectors, asymptomatic infectious carriers,
        past modeling efforts, and so on.
        """,
    goal="Perform searches",
    backstory=textwrap.dedent("""
        Accept list of sub-questions from the query_planner agent and perform
        the necessary searches to answer the questions.
        Perform the tasks in the order given and report the result out.
        Your final answer MUST be a correct response to the original user-query.
    """),
    verbose=True,
    llm=llm,
    # tools=[SqlTools.do_sql_query, RagTools.do_rag_query],
    tools=[CDC_retriever_tool, ABM_retriever_tool, Google_search_tool],
    allow_delegation=True,
)

In [68]:
user_query = "Find a list of gene products in genome 83332.12 which have the same EC number as the EC number for reaction rxn34000 ?"

task1 = Task(
  description=textwrap.dedent(f"""
      If any SQL access is required, you have access to a table that was created
      with this SQL command:
          ```sql
              CREATE TABLE IF NOT EXISTS reactions (
                  REACTION_ID TEXT PRIMARY KEY,
                  EC TEXT
          ```
      Create a description of the sub-queries to handle this user-query:
          {user_query}
      Prepare the correct sub-queries (there may just be one) and pass them
      along to the query_executor.  You should only use the SQL query capability
      if the data is obviously in the SQL tables provided.
  """),
  agent=query_planner
)

task2 = Task(
    description=textwrap.dedent(f"""
        You will receive a set of one or two sub-queries from the query_planner.
        Use those tools to perform SQL queries and/or RAG (text) queries as necessary.
        Your final answer MUST be a correct answer to the original user-query:
            {user_query}
    """),
    agent=query_executor
)

crew = Crew(
    agents=[query_planner, query_executor],
    tasks=[task1, task2],
    verbose=2, # print what tasks are being worked on, can set it to 1 or 2
    process=Process.sequential,
)

result = crew.kickoff()

print("######################")
print(result)